### Imports

In [1]:
import pandas as pd
import numpy as np

import re

from tqdm import tqdm
tqdm.pandas()

from collections import Counter

# nltk library imports
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

In [2]:
### Run nltk download command once to get required stopwords
# nltk.download('punkt')
# nltk.download('stopwords')

stop = stopwords.words('english')
ps=PorterStemmer()

### Functions

In [12]:
def topn_tags(dataFrame ,n):
    """
    This function returns a list of top n tags 
    :param dataFrame: pandas dataFrame
    :param n: integer
    :returns: list of strings
    """
    dataFrame['Tags'] = dataFrame['Tags'].apply(lambda x : x.split(' '))
    
    counter = Counter()
    _ = df['Tags'].apply(counter.update)
    
    unique_tags = counter.most_common(n)       # Top n tags
    tags_df = pd.DataFrame(unique_tags, columns=['Tags','Freq'])
    unique_tags = tags_df.Tags.tolist()
    
    return unique_tags
    

def separate_code_and_body(body):
    """
    This function returns a list of two strings - Code and Description from the given Body content
    :param string: text string of body
    :returns: list of strings containing Code and Body
    """
    code_snippets = re.finditer("<code.*?>(.*?)</code>", body, re.DOTALL)
    code = []
    description = body
    for snip in code_snippets:
        code.append(snip.group())
        body = body.replace(snip.group(),"")
    return [code, body]  


def convert_to_lower(text):
    """
    This function returns a string with lowercase characters
    :param string: text string
    :returns: text string
    """
    return text.lower()

def remove_special_chars(text):
    """
    This function returns a string with filtered special case characters except # and ++ for C# and C++
    :param string: text string
    :returns: text string
    """
    return re.sub(r"[^A-Za-z #++]+",'', text)

def get_list_intersection(x, y):
    """
    This function returns a list which is intersection of two input lists
    :param list: list of strings
    :param list: list of strings
    :returns: Intersection list of strings
    """
    return list(set(x) & set(y))


def remove_html_tags(text):
    """
    This function returns a string after removing html tags from a string
    :param string: text string
    :returns: text string
    """
    clean = re.compile('<.*?>')
    return re.sub(clean,'', text)

def remove_n(text):
    """
    This function returns a string after removing new line char(\n) from a string
    :param string: text string
    :returns: text string
    """
    clean1 = re.compile('\n')
    return re.sub(clean1,'', text)

def get_processed_tags(tags, tag_ids, n):
    """
    This function returns a list of binary values of tags size length for tags represention
    :param list: list of tags
    :param Dictionary: Dict of tags and their ids
    :param int: Number of frequent tags
    :returns: text string
    """
    label = [0]*n
    for tag in tags:
        label[tag_ids[tag]] = 1
    return label

### Read Data

In [5]:
df = pd.read_csv('./../data/processed/train.csv', nrows=10000)

### Number of Most frequent tags taken into consideration
n = 100

### Data Pre - Processing

#### Tags Filtering

In [6]:
# Converting Tags to lower case
df['Tags'] = df['Tags'].progress_apply(lambda x: convert_to_lower(x))

# Drop nan Tag values
df.dropna(subset=['Tags'],inplace=True)

# Filtering to Top 100 tags on Frequency
unique_tags = topn_tags(df,n)

# Filtering the Dataset for top 100 tags
df['Tags'] = df['Tags'].apply(lambda x : get_list_intersection(x,unique_tags))
df = df[df['Tags'].map(lambda d: len(d)) > 0]
df.reset_index(drop=True, inplace=True)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 537724.39it/s]


#### Separate Code from Body

In [7]:
df['Code'] = df['Body'].progress_apply(lambda x : separate_code_and_body(x))
df[['Code','Description']] = pd.DataFrame(df.Code.tolist(), index= df.index)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 7626/7626 [00:00<00:00, 35723.79it/s]


#### Convert to Lower

In [8]:
df['Title'] = df['Title'].progress_apply(lambda x:  convert_to_lower(x))
df['Description'] = df['Description'].progress_apply(lambda x:  convert_to_lower(x))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 7626/7626 [00:00<00:00, 390360.66it/s]


#### Description Processing

In [9]:
df['Description'] = df['Description'].progress_apply(lambda x: remove_html_tags(x))
df['Description'] = df['Description'].progress_apply(lambda x: remove_n(x))
df['Description'] = df['Description'].progress_apply(lambda x: remove_special_chars(x))
df['Description'] = df['Description'].progress_apply(lambda x: word_tokenize(x))
df['Description'] = df['Description'].progress_apply(lambda x: [item for item in x if item not in stop])
df['Description'] = df['Description'].progress_apply(lambda x: [ps.stem(word) for word in x])

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 7626/7626 [00:07<00:00, 1039.15it/s]


#### Title Processing

In [10]:
df['Title'] = df['Title'].progress_apply(lambda x: remove_n(x))
df['Title'] = df['Title'].progress_apply(lambda x: remove_special_chars(x))
df['Title'] = df['Title'].progress_apply(lambda x: word_tokenize(x))
df['Title'] = df['Title'].progress_apply(lambda x: [item for item in x if item not in stop])
df['Title'] = df['Title'].progress_apply(lambda x: [ps.stem(word) for word in x])

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 7626/7626 [00:01<00:00, 7507.52it/s]


#### Merge Title and Body

In [11]:
df['Text'] = df['Title'] + df['Description']
df['Text'] = df['Text'].progress_apply(lambda x: list(set(x)))
df = df[['Text','Tags']]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 7626/7626 [00:00<00:00, 87530.55it/s]


In [13]:
tag_ids = {}

for i in range(len(unique_tags)):
    tag_ids[unique_tags[i]] = i

df['Tags'] = df['Tags'].progress_apply(lambda x : get_processed_tags(x, tag_ids, n))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 7626/7626 [00:00<00:00, 271539.83it/s]


In [ ]:
counter = Counter()
_ = df['Text'].apply(counter.update)